<h1>Transitioning from black-box to XAI: using NLP to provide justification</h1>

We want to leverage GPT2 (it's free, does not need an API key and performs way better than other options: BERT, T5 or BART) to turn SHAP insights into natural language explanations, rather than hand-crafted templates

<h2>Without drift corrected</h2>

In [22]:
model_lstm_gru = load_model('best_model_lstm_gru_after_optuna.h5')

I0000 00:00:1742759600.409753  153916 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2242 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [23]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer

gpt_model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(gpt_model_name)
gpt_model = TFAutoModelForCausalLM.from_pretrained(gpt_model_name)  # TensorFlow version

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [25]:
look_back = X_train.shape[1]     # 90
n_features = X_train.shape[2]

In [26]:
background = X_train[np.random.choice(X_train.shape[0], 300, replace=False)]
background_flat = background.reshape((300, look_back * n_features))

In [27]:
def model_predict_wrapper(X_flat):
    X_reshaped = X_flat.reshape((-1, look_back, n_features))
    return model_lstm_gru.predict(X_reshaped)

In [28]:
explainer = shap.KernelExplainer(model_predict_wrapper, background_flat)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step 


In [29]:
X_test_flat = X_test[:200].reshape((200, look_back * n_features))

In [30]:
shap_values = explainer.shap_values(X_test_flat, nsamples=100)

  0%|                                                                                           | 0/200 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step 


  0%|▍                                                                                  | 1/200 [00:07<25:31,  7.70s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step


  1%|▊                                                                                  | 2/200 [00:14<23:52,  7.23s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████▏| 198/200 [35:13<00:17,  8.88s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step


100%|████████████████████████████████████████████████████████████████████████████████▌| 199/200 [35:22<00:08,  8.93s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step 


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [35:30<00:00, 10.65s/it]


In [36]:
def build_prompt(sequence_input, shap_values, top_k=5):
    mean_shap = np.mean(np.abs(shap_values), axis=0)
    top_indices = np.argsort(mean_shap)[-top_k:][::-1]

    description = ""
    for i in top_indices:
        if i < len(sequence_input) - 1:
            delta = sequence_input[i+1] - sequence_input[i]
            trend = "increased" if delta > 0 else "decreased" if delta < 0 else "stayed the same"
            description += f"- Day {i+1}: Price {trend}\n"

    prompt = (
        "Explain how the provided LSTM-GRU neural network model might use the following 90-day price trends to forecast the next day's closing price. Explain if it's time to BUY, SELL or HOLD on these stocks."
        "The model is looking on a databse which contains the following features: Close Price, Volume, RSI based on Close Price, MACD Line based on Close Price, MACD Signal based on Close Price, MACD Histogram based on Close Price':\n\n"
        f"{description}\nExplanation:"
    )
    return prompt

In [39]:
def generate_gpt2_explanation(prompt, max_length=15000):
    inputs = tokenizer.encode(prompt, return_tensors="tf")
    outputs = gpt_model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [40]:
# Choose one of the 200 SHAP samples
i = int(100)
sequence_sample = sample_input[i]
shap_sample = shap_values[i]

prompt = build_prompt(sequence_sample, shap_sample)
gpt_output = generate_gpt2_explanation(prompt)

print(" GPT-2 Justification of the Model's Prediction:\n")
print(gpt_output)

 GPT-2 Justification of the Model's Prediction:

Explain how the provided LSTM-GRU neural network model might use the following 90-day price trends to forecast the next day's closing price. Explain if it's time to BUY, SELL or HOLD on these stocks.The model is looking on a databse which contains the following features: Close Price, Volume, RSI based on Close Price, MACD Line based on Close Price, MACD Signal based on Close Price, MACD Histogram based on Close Price':

- Day 1: Price decreased

Explanation:
. The expected closing prices will be the same as the previous day.
 "Market" or "market share" is the price of the stock in the market. In this case, the close price is equivalent to the closing stock's price in that market and is therefore equivalent only to that close stock. This can be done by using the "Exchange rate" option in a currency exchange. To convert the MACDs to U.S. dollars, simply use a coin exchange rate of 0.0025 USD. For example, a USD = USD $5.01.


The price cha

For more detailed prompt info, the max_length from generate_gpt2_explanation may be incresed

<h2>With drift corrected</h2>

In [168]:
model_lstm_gru = load_model('best_model_lstm_gru_after_optuna.h5')

In [169]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer

gpt_model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(gpt_model_name)
gpt_model = TFAutoModelForCausalLM.from_pretrained(gpt_model_name)  # TensorFlow version

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [170]:
look_back = X_train.shape[1]     # 90
n_features = X_train.shape[2]

In [171]:
background = X_train[np.random.choice(X_train.shape[0], 300, replace=False)]
background_flat = background.reshape((300, look_back * n_features))

In [180]:
def model_predict_wrapper(X_flat):
    X_reshaped = X_flat.reshape((-1, look_back, n_features))
    residual_preds = model_lstm_gru.predict(X_reshaped).flatten()

    close_idx = 0 
    last_10_closes = X_reshaped[:, -10:, close_idx]  
    rolling_mean = np.mean(last_10_closes, axis=1)  

    final_preds = residual_preds + rolling_mean
    return final_preds.reshape(-1, 1)

In [181]:
explainer = shap.KernelExplainer(model_predict_wrapper, background_flat)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [182]:
X_test_flat = X_test[:200].reshape((200, look_back * n_features))

In [183]:
shap_values = explainer.shap_values(X_test_flat, nsamples=100)

  0%|                                                                                           | 0/200 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step


  0%|▍                                                                                  | 1/200 [00:11<36:42, 11.07s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step


  1%|▊                                                                                  | 2/200 [00:18<29:16,  8.87s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████▏| 198/200 [29:53<00:16,  8.13s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step


100%|████████████████████████████████████████████████████████████████████████████████▌| 199/200 [30:00<00:07,  7.99s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [30:09<00:00,  9.05s/it]


In [195]:
def build_prompt(sequence_input, shap_values, top_k=5):
    mean_shap = np.mean(np.abs(shap_values), axis=0)
    top_indices = np.argsort(mean_shap)[-top_k:][::-1]

    description = ""
    for i in top_indices:
        if i < len(sequence_input) - 1:
            delta = sequence_input[i+1] - sequence_input[i]
            avg_delta = np.mean(delta)  # or use np.sum(np.sign(delta)) for majority trend
    
            trend = "increased" if avg_delta > 0 else "decreased" if avg_delta < 0 else "stayed the same"
            description += f"- Day {i+1}: Price {trend}\n"

    prompt = (
        "Explain how the provided LSTM-GRU neural network model might use the following 90-day price trends to forecast the next day's AAPL closing price. Explain if it's time to BUY, SELL or HOLD on these stocks."
        "The model is looking on a databse which contains the following features: Close Price, Volume, RSI based on Close Price, MACD Line based on Close Price, MACD Signal based on Close Price, MACD Histogram based on Close Price':\n\n"
        f"{description}\nExplanation:"
    )
    return prompt

In [196]:
def generate_gpt2_explanation(prompt, max_length=15000):
    inputs = tokenizer.encode(prompt, return_tensors="tf")
    outputs = gpt_model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [197]:
sample_input = X_test[:200]

In [198]:
# Choose one of the 200 SHAP samples
i = int(100)
sequence_sample = sample_input[i]
shap_sample = shap_values[i]

prompt = build_prompt(sequence_sample, shap_sample)
gpt_output = generate_gpt2_explanation(prompt)

print(" GPT-2 Justification of the Model's Prediction:\n")
print(gpt_output)

 GPT-2 Justification of the Model's Prediction:

Explain how the provided LSTM-GRU neural network model might use the following 90-day price trends to forecast the next day's AAPL closing price. Explain if it's time to BUY, SELL or HOLD on these stocks.The model is looking on a databse which contains the following features: Close Price, Volume, RSI based on Close Price, MACD Line based on Close Price, MACD Signal based on Close Price, MACD Histogram based on Close Price':

- Day 1: Price increased

Explanation: The following two days of data will be used to calculate the price of the first and second day of AAPLA.
. The price is based upon the current market price on the day before the close of trading. For example, the market is trading at $1.00. If the AAPLO price has increased by $2.50 since the start of 2016, then the company will close its AAPLEX stock. On the second and third day, we will calculate its market cap and the underlying market capitalization. We'll also calculate a pr